Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество регрессии не изменится

**Обоснование:** параметры регрессий будут связанны следующим образом:

$$
w1 = A w2
$$
Или
$$
w2 = A^{-1} w1
$$
, где<br>
- $w1$ - параметры обычной регрессии, <br>
- $w2$ - параметры преобразованной регрессии,<br>
- $A$ - обратимая матрица, с помощью которой "шифруем" данные

## 3. Алгоритм преобразования

**Алгоритм**

Создаем обратимую матрицу (назовем ее "матрица-ширафтор") и на нее умножаем исходные данные, за исключением целевого признака. <br>
Напишем функцию, которая создаст матрицу-ширафтор  методом `numpy.random.random_integers()`.  Этот метод почти всегда создает обратимую матрицу, но на всякий случай сделаем проверку: если не получится создать обратимую матрицу, то функция будет вызывать себя пока не создаст обратимую матрицу.

**Обоснование** <br>
Умножение на матрицу, это своего рода, умножение на константу. Отношение коэффициентов моделей будут совпадать <br>
$$
w1_1 : w1_2 : w1_n = w2_1 : w2_2 : w2_n
$$

## 4. Проверка алгоритма

In [4]:
def get_inv_mtx(size_mtx):
    #np.random.seed(42)
    inv_v = np.random.random_integers(5, size=(size_mtx, size_mtx))
    try:
        return inv_v, np.linalg.inv(inv_v) 
    except:
        return get_inv_mtx(size_mtx)

Матрица-ширафтор:

In [5]:
inv_mtx, inv_mtx_2 = get_inv_mtx(data.drop(columns='Страховые выплаты', axis=1).shape[1])

inv_mtx

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: This function is deprecated. Please call randint(1, 5 + 1) instead
  This is separate from the ipykernel package so we can avoid doing imports until


array([[1, 4, 2, 3],
       [3, 2, 1, 2],
       [4, 3, 4, 4],
       [2, 1, 4, 5]])

Защита данных

In [6]:
#Страховые выплаты
data_def = pd.DataFrame(data=data.drop(columns='Страховые выплаты', axis=1).values.dot(inv_mtx), columns=data.columns[:-1])
data_def['Страховые выплаты'] = data['Страховые выплаты']
data_def.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,198526.0,148887.0,198447.0,198490.0,0
1,152140.0,114093.0,152050.0,152097.0,1
2,84087.0,63058.0,84029.0,84058.0,0
3,166867.0,125144.0,166829.0,166852.0,0
4,104485.0,78360.0,104430.0,104459.0,0


## Модель на обычных данных

In [7]:
features_simple = data.drop(columns='Страховые выплаты', axis=1)
target_simple = data['Страховые выплаты']

X_train, X_test, y_train, y_test = train_test_split(features_simple, target_simple, test_size=0.25, random_state=42)

model_simple = LinearRegression()
model_simple.fit(X_train, y_train)
print('коэффициенты обычной модели', model_simple.coef_)
predictions_simple = model_simple.predict(X_test)
print('R2:', r2_score(y_test, predictions_simple))

коэффициенты обычной модели [ 4.92432086e-03  3.51527196e-02 -2.45796619e-07 -1.49140089e-02]
R2: 0.42547785406963123


## Модель на защищенных данных 

In [8]:
features_def = data_def.drop(columns='Страховые выплаты', axis=1)
target_def = data_def['Страховые выплаты']

X_train, X_test, y_train, y_test = train_test_split(features_def, target_def, test_size=0.25, random_state=42)

model_def = LinearRegression()
model_def.fit(X_train, y_train)
print('коэффициенты защищенной модели', model_def.coef_)
predictions_def = model_def.predict(X_test)
print('R2:', r2_score(y_test, predictions_def))

коэффициенты защищенной модели [ 0.01090341  0.00198551 -0.02325668  0.01086408]
R2: 0.42547785406969274


<hr>

Если умножить обратную матрицу-шифратор на коэффициенты модели обычных данных, то получим коэффициенты модели защищенных данных 

In [9]:
print('Коэффициенты защищенной модели:\n', model_def.coef_)
print('Произведение обратной матрицы-шифратора на коэффициенты обычной модели:\n', inv_mtx_2.dot(model_simple.coef_))

Коэффициенты защищенной модели:
 [ 0.01090341  0.00198551 -0.02325668  0.01086408]
Произведение обратной матрицы-шифратора на коэффициенты обычной модели:
 [ 0.01090341  0.00198551 -0.02325668  0.01086408]


In [10]:
print('Коэффициенты обычной модели:\n', model_simple.coef_)
print('Произведение матрицы-шифратора на коэффициенты защищенной модели:\n', inv_mtx.dot(model_def.coef_))

Коэффициенты обычной модели:
 [ 4.92432086e-03  3.51527196e-02 -2.45796619e-07 -1.49140089e-02]
Произведение матрицы-шифратора на коэффициенты защищенной модели:
 [ 4.92432086e-03  3.51527196e-02 -2.45796619e-07 -1.49140089e-02]


<hr>

Отношение коэффициентов моделей:

In [11]:
len_koef = len(model_simple.coef_)
for i in range(len_koef):
    if i + 1 == len_koef: break
    print('Обычная модель', model_def.coef_[i] / model_def.coef_[i+1], '\tЗащищенная модель', model_def.coef_[i] / model_def.coef_[i+1])

Обычная модель 5.49148812047697 	Защищенная модель 5.49148812047697
Обычная модель -0.08537378715375159 	Защищенная модель -0.08537378715375159
Обычная модель -2.1406955932299008 	Защищенная модель -2.1406955932299008


<hr>

Расшифровка данных:

In [12]:
data_def_inv = pd.DataFrame(
    data=data_def.drop(columns='Страховые выплаты', axis=1).values.dot(inv_mtx_2), 
    columns=data.columns[:-1]
)
print('Расшифрованные данные')
data_def_inv.head()

Расшифрованные данные


,Пол,Возраст,Зарплата,Члены семьи
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-4.749243e-12,46.0,38000.0,1.000000e+00
2,-3.181871e-12,29.0,21000.0,-3.698264e-12
3,-2.235434e-13,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,-1.716460e-12


In [13]:
print('Обычные данные')
data.head()

Обычные данные


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


# Вывод

После умножения обычных данных на обратимую матрицу, обучение модели на них скажется на качестве этой модели. <br>
При умножении зашифрованных данных на обратную обратимую матрицу, можно получить обычные данные